# Self-Driving Car Engineer Nanodegree


## Project 5: **Vehicle Detection and Tracking** 
***
Additional notebook:
Shows experimental results with various combinations of classifier parameters.


### Explore Data

#### 1. Imports

In [1]:
# Imports

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import glob
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
import time
from sklearn.svm import LinearSVC
# NOTE: the next import is only valid 
# for scikit-learn version <= 0.17
# if you are using scikit-learn >= 0.18 then use this:
from sklearn.model_selection import train_test_split
# else use this:
#from sklearn.cross_validation import train_test_split

import pickle

from scipy.ndimage.measurements import label


#### 2. Read the training images (vehicle and non-vehicle).

In [2]:
# Read in all the vehicles and non-vehicle images
# Randomly display one example for vehicle and non-vehicle images

path_v = 'project_datasets/vehicles/'
path_nv = 'project_datasets/non-vehicles/'

images_cars = os.listdir(path_v)
images_notcars = os.listdir(path_nv)

cars = []
notcars = []

for image_c in images_cars:
        cars.extend(glob.glob(path_v+image_c+'/*'))
        
for image_nc in images_notcars:
        notcars.extend(glob.glob(path_nv+image_nc+'/*'))
        


### I -  Extract Features

#### 1. Extract binned color feature

In [4]:
# Define a function to compute binned color feature  
# This is scaling down the resolution of an image
def bin_spatial(img, size=(32, 32), vis='False'):
    small_img = cv2.resize(img, size)
    # Use cv2.resize().ravel() to create the feature vector
    feature = small_img.ravel() 
    # Return the feature vector
    if (vis == 'False'):
        return feature
    else:
        return feature,  small_img

#### 2. Extract histogram of color

In [6]:
# Define a function to compute color histogram feature 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256),vis='False'):
    # Compute the histogram of the color channels separately
    # Take histograms in R, G, and B
    rhist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    ghist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    bhist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_feature = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    if (vis == 'False'):
        return hist_feature
    else:
        return hist_feature, rhist, ghist, bhist

#### 3. Extract histogram of gradient (HOG)

In [8]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        # Use skimage.hog() to get both features and a visualization
        features, hog_image = hog(img, 
                                  orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=True, 
                                  feature_vector=False)
                                  #block_norm='L2-Hys')
        return features, hog_image
    else:
        # Use skimage.hog() to get features only
        features = hog(img, 
                       orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=False, 
                       feature_vector=feature_vec)
                       #block_norm='L2-Hys')
        return features

#### 4. Combine and extract features

In [10]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        #image = mpimg.imread(file)
        image = cv2.imread(file) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
                if color_space == 'HSV':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
                elif color_space == 'LUV':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
                elif color_space == 'HLS':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
                elif color_space == 'YUV':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
                elif color_space == 'YCrCb':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)
              

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    



#### 5. Normalize features

In [11]:
# normalize feature
def normalize(car_features, notcar_features):
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    return X, scaled_X

### II - Build and Train a Classifier

#### 1. Linear SVM Classifier

In [14]:
# Train a linear SVM

def classifier(car_features, notcar_features, X, scaled_X):
    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)

    print('Using:',orient,'orientations',pix_per_cell,
        'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))
    # Use a linear SVC 
    svc = LinearSVC()
    # Check the training time for the SVC
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    # Check the prediction time for a single sample
    t=time.time()
    n_predict = 10
    print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
    print('For these',n_predict, 'labels: ', y_test[0:n_predict])
    t2 = time.time()
    print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

#### 2. Combine Feature Extraction, Normalization and Classifier

In [15]:
# Combining and tuning parameters to get best performance
def feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                       spatial_size = (32, 32), 
                       hist_bins=32,  
                       orient = 9,
                       pix_per_cell = 8,
                       cell_per_block = 2,
                       hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                       spatial_feat=True,
                       hist_feat=True,
                       hog_feat=True): 
 
    t=time.time()
 

    car_features = extract_features(cars, 
                                color_space=color_space, 
                                spatial_size=spatial_size,
                                hist_bins=hist_bins, 
                                orient=orient, 
                                pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel,
                                spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, 
                                hog_feat=hog_feat)



    notcar_features = extract_features(notcars, 
                                   color_space=color_space, 
                                   spatial_size=spatial_size,
                                   hist_bins=hist_bins, 
                                   orient=orient, 
                                   pix_per_cell=pix_per_cell, 
                                   cell_per_block=cell_per_block, 
                                   hog_channel=hog_channel,
                                   spatial_feat=spatial_feat, 
                                   hist_feat=hist_feat, 
                                   hog_feat=hog_feat)

    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to extract features...')

    X, scaled_X = normalize(car_features, notcar_features)

    classifier(car_features, notcar_features, X, scaled_X)


#### 3.  Experiment with parameters

#### (i) Exploring Color Space

In [16]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


71.29 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
4.84 Seconds to train SVC...
Test Accuracy of SVC =  0.9944
My SVC predicts:  [ 1.  1.  0.  1.  0.  0.  1.  0.  1.  1.]
For these 10 labels:  [ 0.  1.  0.  1.  0.  0.  1.  0.  1.  1.]
0.0009 Seconds to predict 10 labels with SVC


In [17]:
feature_classifier(color_space = 'YUV', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


84.11 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
6.31 Seconds to train SVC...
Test Accuracy of SVC =  0.9927
My SVC predicts:  [ 0.  0.  0.  0.  1.  1.  1.  0.  0.  0.]
For these 10 labels:  [ 0.  0.  1.  0.  1.  1.  1.  0.  0.  0.]
0.00093 Seconds to predict 10 labels with SVC


In [18]:
feature_classifier(color_space = 'HLS', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True)

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


86.07 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
39.12 Seconds to train SVC...
Test Accuracy of SVC =  0.993
My SVC predicts:  [ 1.  0.  1.  1.  1.  1.  0.  0.  1.  0.]
For these 10 labels:  [ 1.  0.  1.  1.  1.  1.  0.  0.  1.  0.]
0.00083 Seconds to predict 10 labels with SVC


In [19]:
feature_classifier(color_space = 'LUV', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True)

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


79.57 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
18.49 Seconds to train SVC...
Test Accuracy of SVC =  0.9944
My SVC predicts:  [ 1.  1.  1.  1.  1.  1.  0.  0.  0.  1.]
For these 10 labels:  [ 1.  1.  1.  1.  1.  1.  0.  0.  0.  1.]
0.00088 Seconds to predict 10 labels with SVC


In [20]:
feature_classifier(color_space = 'HSV', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


75.07 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
4.26 Seconds to train SVC...
Test Accuracy of SVC =  0.9924
My SVC predicts:  [ 1.  1.  0.  1.  0.  1.  1.  0.  1.  1.]
For these 10 labels:  [ 1.  1.  0.  1.  0.  1.  1.  0.  1.  1.]
0.00108 Seconds to predict 10 labels with SVC


In [21]:
feature_classifier(color_space = 'RGB', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


70.05 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
26.35 Seconds to train SVC...
Test Accuracy of SVC =  0.9856
My SVC predicts:  [ 0.  0.  1.  1.  1.  1.  1.  1.  1.  0.]
For these 10 labels:  [ 0.  0.  1.  1.  1.  1.  1.  1.  1.  0.]
0.001 Seconds to predict 10 labels with SVC


#### (ii) Exploring spatial_size 

In [22]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (64, 64), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


72.67 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 17676
23.64 Seconds to train SVC...
Test Accuracy of SVC =  0.9904
My SVC predicts:  [ 0.  0.  1.  1.  0.  0.  0.  0.  1.  1.]
For these 10 labels:  [ 0.  0.  1.  1.  0.  0.  0.  0.  1.  1.]
0.00208 Seconds to predict 10 labels with SVC


In [23]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


83.36 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
5.04 Seconds to train SVC...
Test Accuracy of SVC =  0.991
My SVC predicts:  [ 1.  0.  1.  1.  1.  0.  0.  1.  0.  1.]
For these 10 labels:  [ 1.  0.  1.  1.  1.  0.  0.  1.  0.  1.]
0.00156 Seconds to predict 10 labels with SVC


In [24]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (16, 16), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


73.8 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6156
12.05 Seconds to train SVC...
Test Accuracy of SVC =  0.9921
My SVC predicts:  [ 0.  1.  0.  1.  1.  0.  1.  0.  0.  0.]
For these 10 labels:  [ 0.  1.  0.  1.  1.  0.  1.  0.  0.  0.]
0.00085 Seconds to predict 10 labels with SVC


In [25]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (8, 8), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


69.76 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 5580
15.86 Seconds to train SVC...
Test Accuracy of SVC =  0.9918
My SVC predicts:  [ 0.  1.  1.  1.  0.  0.  0.  1.  0.  1.]
For these 10 labels:  [ 0.  1.  1.  1.  0.  0.  0.  1.  0.  1.]
0.00111 Seconds to predict 10 labels with SVC


#### (iii) Exploring orient 

In [26]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 10,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


71.34 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 9048
3.91 Seconds to train SVC...
Test Accuracy of SVC =  0.9918
My SVC predicts:  [ 1.  1.  0.  1.  1.  0.  1.  0.  0.  0.]
For these 10 labels:  [ 1.  1.  0.  1.  1.  0.  1.  0.  0.  0.]
0.00105 Seconds to predict 10 labels with SVC


In [27]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 8,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


71.15 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 7872
10.82 Seconds to train SVC...
Test Accuracy of SVC =  0.9944
My SVC predicts:  [ 1.  1.  1.  0.  0.  0.  0.  0.  1.  0.]
For these 10 labels:  [ 1.  1.  1.  0.  0.  0.  0.  0.  1.  0.]
0.00236 Seconds to predict 10 labels with SVC


#### (iv) Exploring pix_per_cell

In [28]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 9,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


56.49 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 7056
14.64 Seconds to train SVC...
Test Accuracy of SVC =  0.9932
My SVC predicts:  [ 0.  0.  1.  0.  0.  0.  0.  1.  1.  0.]
For these 10 labels:  [ 0.  0.  1.  0.  0.  0.  0.  1.  1.  0.]
0.00108 Seconds to predict 10 labels with SVC


In [29]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 7,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


79.86 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 10080
6.07 Seconds to train SVC...
Test Accuracy of SVC =  0.9941
My SVC predicts:  [ 1.  1.  1.  1.  1.  0.  0.  1.  0.  1.]
For these 10 labels:  [ 1.  1.  1.  1.  1.  0.  0.  1.  0.  1.]
0.00135 Seconds to predict 10 labels with SVC


#### (v) Exploring cell_per_block

In [30]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 4,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


61.15 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 13968
12.47 Seconds to train SVC...
Test Accuracy of SVC =  0.9913
My SVC predicts:  [ 1.  0.  0.  0.  0.  0.  1.  0.  1.  1.]
For these 10 labels:  [ 1.  0.  0.  0.  0.  0.  1.  0.  1.  1.]
0.00098 Seconds to predict 10 labels with SVC


In [31]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 8,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


51.08 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 4896
11.36 Seconds to train SVC...
Test Accuracy of SVC =  0.9904
My SVC predicts:  [ 1.  1.  1.  0.  1.  0.  0.  1.  1.  0.]
For these 10 labels:  [ 1.  1.  1.  0.  1.  0.  0.  1.  1.  0.]
0.00083 Seconds to predict 10 labels with SVC


#### (vi) Exploring hog_channel

In [32]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 0, # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


33.15 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 4932
12.59 Seconds to train SVC...
Test Accuracy of SVC =  0.987
My SVC predicts:  [ 0.  0.  1.  1.  1.  1.  1.  1.  0.  0.]
For these 10 labels:  [ 0.  0.  1.  1.  1.  1.  1.  1.  0.  0.]
0.00141 Seconds to predict 10 labels with SVC


In [33]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 1, # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


32.95 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 4932
13.73 Seconds to train SVC...
Test Accuracy of SVC =  0.9704
My SVC predicts:  [ 1.  0.  0.  1.  0.  1.  1.  0.  0.  0.]
For these 10 labels:  [ 1.  0.  0.  1.  0.  1.  1.  0.  0.  0.]
0.00082 Seconds to predict 10 labels with SVC


In [34]:
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 2, # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 

/Users/preetichauhan/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


29.19 Seconds to extract features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 4932
14.3 Seconds to train SVC...
Test Accuracy of SVC =  0.9631
My SVC predicts:  [ 0.  1.  0.  0.  0.  0.  1.  0.  1.  0.]
For these 10 labels:  [ 0.  1.  0.  0.  0.  0.  1.  0.  1.  0.]
0.00088 Seconds to predict 10 labels with SVC


#### Summary

Following combinations give the best results (Test Accuracy of SVC =  0.9944):

    
```
feature_classifier(color_space = 'YCrCb', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True) 
```

OR

```
feature_classifier(color_space = 'LUV', # Can be RGB, HSV, LUV, HLS, YUV, YCrCb 
                   spatial_size = (32, 32), 
                   hist_bins=32,  
                   orient = 9,
                   pix_per_cell = 8,
                   cell_per_block = 2,
                   hog_channel = 'ALL', # Can be 0, 1, 2, or "ALL"
                   spatial_feat=True,
                   hist_feat=True,
                   hog_feat=True)
```